## Projeto Detecção de Fraude de Cartão de Crédito ( PIX?)

In [44]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
plt.style.use('default')
import sklearn 
import imblearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score

In [2]:
fraud_detection = pd.read_csv('creditcard.csv')
fraud_detection.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
#fraud_detection.info()

In [4]:
fraud_detection.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
#fraud_detection.info()

In [6]:
# Tratar valores ausentes
fraud_detection.fillna(fraud_detection.mean(), inplace=True)

In [7]:
#fraud_detection.info()

In [8]:
# Normalizando os dados
fraud_detection[["Time", "Amount"]] = (fraud_detection[["Time", "Amount"]] - fraud_detection[["Time", "Amount"]].mean()) / fraud_detection[["Time", "Amount"]].std()

In [9]:
#fraud_detection.info()

In [10]:
#fraud_detection.head()

In [11]:
fraud_detection.shape

(284807, 31)

In [12]:
# Checagem das colunas
fraud_detection.columns[:-1].values

array(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27',
       'V28', 'Amount'], dtype=object)

In [13]:
fraud_detection.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000
mean,-1.149614e-16,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,2.308662e-13,0.001727
std,1.000000e+00,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,1.000000e+00,0.041527
min,-1.996580e+00,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,-3.532288e-01,0.000000
25%,-8.552105e-01,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,-3.308395e-01,0.000000
50%,-2.131450e-01,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,-2.652710e-01,0.000000
75%,9.372158e-01,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,-4.471699e-02,0.000000
max,1.642055e+00,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,1.023621e+02,1.000000


### Analisando os dados

Há várias técnicas que você pode utilizar para lidar com desbalanceamento de dados. Algumas das técnicas mais comuns incluem:

Oversampling: consiste em criar cópias das instâncias da classe minoritária para aumentar a sua representatividade no conjunto de dados.

Undersampling: consiste em remover aleatoriamente instâncias da classe majoritária para diminuir a sua representatividade no conjunto de dados.

Synthetic Minority Over-sampling Technique (SMOTE): essa técnica cria novas instâncias sintéticas da classe minoritária através da combinação linear de instâncias existentes.

Adição de peso às classes: consiste em adicionar um peso maior à classe minoritária no cálculo da função de custo do modelo.

Ensemble de modelos: pode-se utilizar técnicas de ensemble para combinar vários modelos que foram treinados em subconjuntos do conjunto de dados. Essa técnica pode melhorar o desempenho do modelo em problemas de desbalanceamento.

Mudança da métrica de avaliação: em vez de utilizar métricas como acurácia, pode-se utilizar outras métricas que levam em conta o desbalanceamento do conjunto de dados, como F1-score, AUC-ROC, entre outras.

Essas são apenas algumas das técnicas que podem ser utilizadas para lidar com desbalanceamento de dados. A escolha da técnica mais apropriada depende do problema em questão e da disponibilidade de dados. É importante avaliar o desempenho do modelo em diferentes técnicas para determinar qual delas é a mais adequada para o seu problema.

###  Note que a normalização dos dados é opcional, mas é recomendável para muitos modelos de aprendizagem de máquina. A conversão das colunas de tipo objeto para categórico também é opcional, mas pode ser benéfico para alguns modelos.





In [14]:
# Contagem de grupos
fraud_detection.groupby(by=['Class'])['Class'].count()

Class
0    284315
1       492
Name: Class, dtype: int64

## OVERSAMPLING(SMOTE)

Se o oversampling for feito antes da validação cruzada, o oversampling da classe minoritária pode resultar em um problema de overfitting.

Erro de sobreajuste de validação cruzada:

Overfitting durante a validação cruzada:
Em nossa análise de subamostra, quero mostrar um erro comum que cometi e que quero compartilhar com todos vocês. É simples, se você deseja subamostrar ou superamostrar seus dados, não deve fazê-lo antes da validação cruzada. Porque você estará influenciando diretamente o conjunto de validação antes de implementar a validação cruzada, causando um problema de "vazamento de dados". Na seção a seguir, você verá pontuações surpreendentes de precisão e recuperação, mas, na realidade, nossos dados estão superajustados!

O caminho errado:

Como mencionado anteriormente, se pegarmos a classe minoritária ("Fraude") em nosso caso, e criarmos os pontos sintéticos antes da validação cruzada, teremos uma certa influência no "conjunto de validação" do processo de validação cruzada. Lembre-se de como funciona a validação cruzada, vamos suponha que estamos dividindo os dados em 5 lotes, 4/5 do conjunto de dados será o conjunto de treinamento enquanto 1/5 será o conjunto de validação. O conjunto de teste não deve ser tocado! Por esse motivo, temos que fazer a criação de pontos de dados sintéticos "durante" a validação cruzada e não antes, como abaixo:

A Maneira Certa:

Como você pode ver acima, o SMOTE ocorre "durante" a validação cruzada e não "antes" do processo de validação cruzada. Os dados sintéticos são criados apenas para o conjunto de treinamento sem afetar o conjunto de validação.

In [15]:
from collections import Counter
import time

In [16]:
X = fraud_detection.drop('Class', axis=1)
y = fraud_detection['Class']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=11)

In [18]:
# Imprime a quantidade de classes 0 e 1 antes do SMOTE
print('Antes do SMOTE:', Counter(y_train))

Antes do SMOTE: Counter({0: 227451, 1: 394})


## Regressão Logística

In [19]:
start_time = time.time()
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['scaler', MinMaxScaler()],
                                ['classifier', LogisticRegression(random_state=11,
                                                                  max_iter=1000)]])

In [20]:
stratified_kfold = StratifiedKFold(n_splits=3,
                                       shuffle=True,
                                       random_state=11)

In [21]:
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           n_jobs=-1)

In [22]:
#grid_search.fit(X_train, y_train)
#lr_mean_score = grid_search.best_score_
#test_score = grid_search.score(X_test, y_test)
#print(f'Cross-validation score: {lr_mean_score}\nTest score: {test_score}')

In [23]:
grid_search.fit(X_train, y_train)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)
y_pred_lr = grid_search.predict(X_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Cross-validation score: 0.9814658870131886
Test score: 0.9637525768607949


In [25]:
print("Melhores parâmetros: ", grid_search.best_params_)
print("Melhores hiper-parâmetros: ", grid_search.best_estimator_)
print("Melhor pontuação de validação cruzada: ", cv_score)

execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

print("--- %d minutes %d seconds ---" % (minutes, seconds))

Melhores parâmetros:  {'classifier__C': 100}
Melhores hiper-parâmetros:  Pipeline(steps=[('smote', SMOTE(random_state=11)), ('scaler', MinMaxScaler()),
                ['classifier',
                 LogisticRegression(C=100, max_iter=1000, random_state=11)]])
Melhor pontuação de validação cruzada:  0.9814658870131886
--- 7 minutes 53 seconds ---


In [26]:
# Printing the classification report
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962



No trecho de código acima, usamos SMOTE como parte de um pipeline. Este pipeline não é um pipeline 'Scikit-Learn', mas um pipeline 'imblearn'. Como o SMOTE não possui um método 'fit_transform', não podemos usá-lo com o pipeline 'Scikit-Learn'.

A partir dos resultados dos dois métodos acima, não conseguimos ver uma grande diferença entre as pontuações de validação cruzada dos dois métodos. No entanto, o segundo método resultou em uma pontuação de validação cruzada ligeiramente mais próxima da pontuação do teste em comparação com o primeiro método. Isso pode ter acontecido apenas por acaso e também porque o conjunto de dados não é altamente desequilibrado.

### Apliquei o smote separadamente só para mostrar a quantidade de labels depois do smote

In [27]:
smote = SMOTE(random_state=11)
X_resampled, y_resampled = smote.fit_resample(X, y)
print('Depois do SMOTE:', Counter(y_resampled))

Depois do SMOTE: Counter({0: 284315, 1: 284315})


## Árvore de Decisão

In [34]:
start_time = time.time()
pipeline_dt = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                   ['scaler', MinMaxScaler()],
                                   ['classifier', DecisionTreeClassifier(random_state=11)]])

param_grid_dt = {'classifier__max_depth': [3, 5, 7, None],
                 'classifier__min_samples_split': [2, 5, 10],
                 'classifier__min_samples_leaf': [1, 2, 4]}

grid_search_dt = GridSearchCV(estimator=pipeline_dt,
                              param_grid=param_grid_dt,
                              scoring='roc_auc',
                              cv=stratified_kfold,
                              n_jobs=-1)


In [35]:
grid_search_dt.fit(X_train, y_train)
cv_score = grid_search_dt.best_score_
test_score = grid_search_dt.score(X_test, y_test)
y_pred_dt = grid_search_dt.predict(X_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Cross-validation score: 0.9382844712383688
Test score: 0.9761331727401145


In [70]:
y_pred_dt = grid_search_dt.predict(X_test)

In [36]:
print("Melhores parâmetros: ", grid_search_dt.best_params_)
print("Melhores hiper-parâmetros: ", grid_search_dt.best_estimator_)
print("Melhor pontuação de validação cruzada: ", cv_score)

execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

print("--- %d minutes %d seconds ---" % (minutes, seconds))

Melhores parâmetros:  {'classifier__max_depth': 3, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Melhores hiper-parâmetros:  Pipeline(steps=[('smote', SMOTE(random_state=11)), ('scaler', MinMaxScaler()),
                ['classifier',
                 DecisionTreeClassifier(max_depth=3, random_state=11)]])
Melhor pontuação de validação cruzada:  0.9382844712383688
--- 2 minutes 4 seconds ---


In [37]:
# Printing the classification report
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     56864
           1       0.04      0.92      0.07        98

    accuracy                           0.96     56962
   macro avg       0.52      0.94      0.52     56962
weighted avg       1.00      0.96      0.98     56962



## KNN:

In [47]:
pipeline_knn = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                    ['scaler', MinMaxScaler()],
                                    ['classifier', KNeighborsClassifier()]])
                                    
param_grid_knn = {'classifier__n_neighbors': [3, 5, 7, 9, 11],
                  'classifier__weights': ['uniform', 'distance']}

grid_search_knn = GridSearchCV(estimator=pipeline_knn,
                               param_grid=param_grid_knn,
                               scoring='roc_auc',
                               cv=stratified_kfold,
                               n_jobs=-1)


In [48]:
grid_search_knn.fit(X_train, y_train)
cv_score = grid_search_knn.best_score_
test_score = grid_search_knn.score(X_test, y_test)
y_pred_knn = grid_search_knn.predict(X_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Cross-validation score: nan
Test score: 0.9382746732626647


In [49]:
print("Melhores parâmetros: ", grid_search_knn.best_params_)
print("Melhores hiper-parâmetros: ", grid_search_knn.best_estimator_)
print("Melhor pontuação de validação cruzada: ", cv_score)

execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

print("--- %d minutes %d seconds ---" % (minutes, seconds))

Melhores parâmetros:  {'classifier__n_neighbors': 3, 'classifier__weights': 'uniform'}
Melhores hiper-parâmetros:  Pipeline(steps=[('smote', SMOTE(random_state=11)), ('scaler', MinMaxScaler()),
                ['classifier', KNeighborsClassifier(n_neighbors=3)]])
Melhor pontuação de validação cruzada:  nan
--- 613 minutes 22 seconds ---


In [50]:
# Printing the classification report
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.49      0.87      0.63        98

    accuracy                           1.00     56962
   macro avg       0.75      0.93      0.81     56962
weighted avg       1.00      1.00      1.00     56962



## Random Forest Tree:

In [51]:
pipeline_rf = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                   ['scaler', MinMaxScaler()],
                                   ['classifier', RandomForestClassifier(random_state=11)]])

param_grid_rf = {'classifier__n_estimators': [50, 100, 200, 500],
                 'classifier__max_depth': [3, 5, 7, None],
                 'classifier__criterion': ['gini', 'entropy']}

grid_search_rf = GridSearchCV(estimator=pipeline_rf,
                              param_grid=param_grid_rf,
                              scoring='roc_auc',
                              cv=stratified_kfold,
                              n_jobs=-1)

In [52]:
grid_search_rf.fit(X_train, y_train)
cv_score = grid_search_rf.best_score_
test_score = grid_search_rf.score(X_test, y_test)
y_pred_rf = grid_search_rf.predict(X_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Cross-validation score: 0.983303680816686
Test score: 0.9822368874392751


In [53]:
print("Melhores parâmetros: ", grid_search_rf.best_params_)
print("Melhores hiper-parâmetros: ", grid_search_rf.best_estimator_)
print("Melhor pontuação de validação cruzada: ", cv_score)

execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

print("--- %d minutes %d seconds ---" % (minutes, seconds))

Melhores parâmetros:  {'classifier__criterion': 'gini', 'classifier__max_depth': 7, 'classifier__n_estimators': 500}
Melhores hiper-parâmetros:  Pipeline(steps=[('smote', SMOTE(random_state=11)), ('scaler', MinMaxScaler()),
                ['classifier',
                 RandomForestClassifier(max_depth=7, n_estimators=500,
                                        random_state=11)]])
Melhor pontuação de validação cruzada:  0.983303680816686
--- 671 minutes 45 seconds ---


In [54]:
# Printing the classification report
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.37      0.89      0.52        98

    accuracy                           1.00     56962
   macro avg       0.69      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962



## XG Boost:

In [55]:
pipeline_xgb = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                    ['scaler', MinMaxScaler()],
                                    ['classifier', XGBClassifier(random_state=11)]])

param_grid_xgb = {'classifier__max_depth': [3, 5, 7, 9],
                  'classifier__learning_rate': [0.01, 0.1, 0.2],
                  'classifier__n_estimators': [50, 100, 200, 500]}

grid_search_xgb = GridSearchCV(estimator=pipeline_xgb,
                               param_grid=param_grid_xgb,
                               scoring='roc_auc',
                               cv=stratified_kfold,
                               n_jobs=-1)


In [56]:
grid_search_xgb.fit(X_train, y_train)
cv_score = grid_search_xgb.best_score_
test_score = grid_search_xgb.score(X_test, y_test)
y_pred_xgb = grid_search_xgb.predict(X_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

[11:42:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.9799500893636436
Test score: 0.969099024668956


In [57]:
print("Melhores parâmetros: ", grid_search_xgb.best_params_)
print("Melhores hiper-parâmetros: ", grid_search_xgb.best_estimator_)
print("Melhor pontuação de validação cruzada: ", cv_score)

execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

print("--- %d minutes %d seconds ---" % (minutes, seconds))

Melhores parâmetros:  {'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 500}
Melhores hiper-parâmetros:  Pipeline(steps=[('smote', SMOTE(random_state=11)), ('scaler', MinMaxScaler()),
                ['classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=500,
                               n_jobs=16, num_parallel_tree=1, predictor='auto',
                               random_state=11, reg_alpha=0, reg_lambda=1,
                   

In [58]:
# Printing the classification report
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.09      0.91      0.17        98

    accuracy                           0.98     56962
   macro avg       0.55      0.95      0.58     56962
weighted avg       1.00      0.98      0.99     56962



# Resultados

In [28]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [31]:
cv_score # média do score da regressão logística
precision = precision_score(y_train, grid_search.predict(X_train))
recall = recall_score(y_train, grid_search.predict(X_train))
f1 = f1_score(y_train, grid_search.predict(X_train))
ballance_accuracy = balanced_accuracy_score(y_train, grid_search.predict(X_train))
accuracy = accuracy_score(y_train, grid_search.predict(X_train))
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_train, grid_search.predict(X_train), average='macro')
roc_auc = roc_auc_score(y_train, grid_search.predict(X_train))
confusion_matrix = metrics.confusion_matrix(y_train, grid_search.predict(X_train))
classification = metrics.classification_report(y_train, grid_search.predict(X_train))
print()
print('============================== {} - AVALIAÇÃO - REGRESSãO LOGISTICA ==============================')
print()
print ("MEAN SCORE:" "\n", cv_score)
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - REGRESSãO LOGISTICA ==============================

MEAN SCORE:
 0.9814658870131886

PRECISãO:
 0.05976225370460837

RECALL:
 0.9314720812182741

F1:
 0.11231828615149196

ACURÁRIA BALANCEADA:
 0.9530431968757616

ACURÁCIA:
 0.974539709012706

PRECISãO MACRO:
 0.5298202348521598

RECALL MACRO:
 0.9530431968757616

F1 MACRO:
 0.5497014614290984

ROC/AUC:
 0.9530431968757616

MATRIZ DE CONFUSÃO:
 [[221677   5774]
 [    27    367]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99    227451
           1       0.06      0.93      0.11       394

    accuracy                           0.97    227845
   macro avg       0.53      0.95      0.55    227845
weighted avg       1.00      0.97      0.99    227845




In [59]:
cv_score # média do score 
precision = precision_score(y_train, grid_search_dt.predict(X_train))
recall = recall_score(y_train, grid_search_dt.predict(X_train))
f1 = f1_score(y_train, grid_search_dt.predict(X_train))
ballance_accuracy = balanced_accuracy_score(y_train, grid_search_dt.predict(X_train))
accuracy = accuracy_score(y_train, grid_search_dt.predict(X_train))
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_train, grid_search_dt.predict(X_train), average='macro')
roc_auc = roc_auc_score(y_train, grid_search_dt.predict(X_train))
confusion_matrix = metrics.confusion_matrix(y_train, grid_search_dt.predict(X_train))
classification = metrics.classification_report(y_train, grid_search_dt.predict(X_train))

print()
print('============================== {} - AVALIAÇÃO - ÁRVORE DE DECISÃO ==============================')
print()
print ("MEAN SCORE:" "\n", cv_score)
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - ÁRVORE DE DECISÃO ==============================

MEAN SCORE:
 0.9799500893636436

PRECISãO:
 0.036050470658922494

RECALL:
 0.9137055837563451

F1:
 0.06936416184971099

ACURÁRIA BALANCEADA:
 0.9356921902540865

ACURÁCIA:
 0.957602756259738

PRECISãO MACRO:
 0.5179472032077679

RECALL MACRO:
 0.9356921902540865

F1 MACRO:
 0.5238357042434426

ROC/AUC:
 0.9356921902540866

MATRIZ DE CONFUSÃO:
 [[217825   9626]
 [    34    360]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98    227451
           1       0.04      0.91      0.07       394

    accuracy                           0.96    227845
   macro avg       0.52      0.94      0.52    227845
weighted avg       1.00      0.96      0.98    227845




In [60]:
cv_score # média do score 
precision = precision_score(y_train, grid_search_knn.predict(X_train))
recall = recall_score(y_train, grid_search_knn.predict(X_train))
f1 = f1_score(y_train, grid_search_knn.predict(X_train))
ballance_accuracy = balanced_accuracy_score(y_train, grid_search_knn.predict(X_train))
accuracy = accuracy_score(y_train, grid_search_knn.predict(X_train))
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_train, grid_search_knn.predict(X_train), average='macro')
roc_auc = roc_auc_score(y_train, grid_search_knn.predict(X_train))
confusion_matrix = metrics.confusion_matrix(y_train, grid_search_knn.predict(X_train))
classification = metrics.classification_report(y_train, grid_search_knn.predict(X_train))
print()
print('============================== {} - AVALIAÇÃO - KNN ==============================')
print()
print ("MEAN SCORE:" "\n", cv_score)
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - KNN ==============================

MEAN SCORE:
 0.9799500893636436

PRECISãO:
 0.7023172905525846

RECALL:
 1.0

F1:
 0.8251308900523561

ACURÁRIA BALANCEADA:
 0.9996328879626821

ACURÁCIA:
 0.9992670455792315

PRECISãO MACRO:
 0.8511586452762923

RECALL MACRO:
 0.9996328879626821

F1 MACRO:
 0.912381821597148

ROC/AUC:
 0.999632887962682

MATRIZ DE CONFUSÃO:
 [[227284    167]
 [     0    394]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.70      1.00      0.83       394

    accuracy                           1.00    227845
   macro avg       0.85      1.00      0.91    227845
weighted avg       1.00      1.00      1.00    227845




In [61]:
cv_score # média do score 
precision = precision_score(y_train, grid_search_rf.predict(X_train))
recall = recall_score(y_train, grid_search_rf.predict(X_train))
f1 = f1_score(y_train, grid_search_rf.predict(X_train))
ballance_accuracy = balanced_accuracy_score(y_train, grid_search_rf.predict(X_train))
accuracy = accuracy_score(y_train, grid_search_rf.predict(X_train))
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_train, grid_search_rf.predict(X_train), average='macro')
roc_auc = roc_auc_score(y_train, grid_search_rf.predict(X_train))
confusion_matrix = metrics.confusion_matrix(y_train, grid_search_rf.predict(X_train))
classification = metrics.classification_report(y_train, grid_search_rf.predict(X_train))
print()
print('============================== {} - AVALIAÇÃO - RANDOM FOREST TREE ==============================')
print()
print ("MEAN SCORE:" "\n", cv_score)
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - RANDOM FOREST TREE ==============================

MEAN SCORE:
 0.9799500893636436

PRECISãO:
 0.4067245119305857

RECALL:
 0.9517766497461929

F1:
 0.5699088145896657

ACURÁRIA BALANCEADA:
 0.9746858680802049

ACURÁCIA:
 0.9975158550769163

PRECISãO MACRO:
 0.7033203915443218

RECALL MACRO:
 0.9746858680802049

F1 MACRO:
 0.784331572361496

ROC/AUC:
 0.9746858680802049

MATRIZ DE CONFUSÃO:
 [[226904    547]
 [    19    375]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.41      0.95      0.57       394

    accuracy                           1.00    227845
   macro avg       0.70      0.97      0.78    227845
weighted avg       1.00      1.00      1.00    227845




In [62]:
cv_score # média do score 
precision = precision_score(y_train, grid_search_xgb.predict(X_train))
recall = recall_score(y_train, grid_search_xgb.predict(X_train))
f1 = f1_score(y_train, grid_search_xgb.predict(X_train))
ballance_accuracy = balanced_accuracy_score(y_train, grid_search_xgb.predict(X_train))
accuracy = accuracy_score(y_train, grid_search_xgb.predict(X_train))
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_train, grid_search_xgb.predict(X_train), average='macro')
roc_auc = roc_auc_score(y_train, grid_search_xgb.predict(X_train))
confusion_matrix = metrics.confusion_matrix(y_train, grid_search_xgb.predict(X_train))
classification = metrics.classification_report(y_train, grid_search_xgb.predict(X_train))
print()
print('============================== {} - AVALIAÇÃO - XG BOOST ==============================')
print()
print ("MEAN SCORE:" "\n", cv_score)
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - XG BOOST ==============================

MEAN SCORE:
 0.9799500893636436

PRECISãO:
 0.09460496036819228

RECALL:
 0.9390862944162437

F1:
 0.17189314750290363

ACURÁRIA BALANCEADA:
 0.9617590530515783

ACURÁCIA:
 0.9843533981434748

PRECISãO MACRO:
 0.5472488929664338

RECALL MACRO:
 0.9617590530515783

F1 MACRO:
 0.5819976166527445

ROC/AUC:
 0.9617590530515783

MATRIZ DE CONFUSÃO:
 [[223910   3541]
 [    24    370]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99    227451
           1       0.09      0.94      0.17       394

    accuracy                           0.98    227845
   macro avg       0.55      0.96      0.58    227845
weighted avg       1.00      0.98      0.99    227845




## Validação

In [32]:
precision = precision_score(y_test, y_pred_lr)
recall = recall_score(y_test, y_pred_lr)
f1 = f1_score(y_test, y_pred_lr)
ballance_accuracy = balanced_accuracy_score(y_test, y_pred_lr)
accuracy = accuracy_score(y_test, y_pred_lr)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_lr, average='macro')
roc_auc = roc_auc_score(y_test, y_pred_lr)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_lr)
classification = metrics.classification_report(y_test, y_pred_lr)
print()
print('============================== {} - AVALIAÇÃO - REGRESSãO LOGÍSTICA ==============================')
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - REGRESSãO LOGÍSTICA ==============================

PRECISãO:
 0.06

RECALL:
 0.9183673469387755

F1:
 0.11264080100125157

ACURÁRIA BALANCEADA:
 0.946785671218403

ACURÁCIA:
 0.9751062111583161

PRECISãO MACRO:
 0.5299278785474739

RECALL MACRO:
 0.946785671218403

F1 MACRO:
 0.5500084157419769

ROC/AUC:
 0.9467856712184028

MATRIZ DE CONFUSÃO:
 [[55454  1410]
 [    8    90]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962




In [71]:
precision = precision_score(y_test, y_pred_dt)
recall = recall_score(y_test, y_pred_dt)
f1 = f1_score(y_test, y_pred_dt)
ballance_accuracy = balanced_accuracy_score(y_test, y_pred_dt)
accuracy = accuracy_score(y_test, y_pred_dt)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
roc_auc = roc_auc_score(y_test, y_pred_dt)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_dt)
classification = metrics.classification_report(y_test, y_pred_dt)
print()
print('============================== {} - AVALIAÇÃO - ARVORE DE DECISÃO ==============================')
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - ARVORE DE DECISÃO ==============================

PRECISãO:
 0.03523884103367267

RECALL:
 0.9183673469387755

F1:
 0.06787330316742081

ACURÁRIA BALANCEADA:
 0.9375179447130568

ACURÁCIA:
 0.9566026473789544

PRECISãO MACRO:
 0.5175459019166304

RECALL MACRO:
 0.9375179447130568

F1 MACRO:
 0.522828735845699

ROC/AUC:
 0.9375179447130568

MATRIZ DE CONFUSÃO:
 [[54400  2464]
 [    8    90]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98     56864
           1       0.04      0.92      0.07        98

    accuracy                           0.96     56962
   macro avg       0.52      0.94      0.52     56962
weighted avg       1.00      0.96      0.98     56962




In [66]:
precision = precision_score(y_test, y_pred_knn)
recall = recall_score(y_test, y_pred_knn)
f1 = f1_score(y_test, y_pred_knn)
ballance_accuracy = balanced_accuracy_score(y_test, y_pred_knn)
accuracy = accuracy_score(y_test, y_pred_knn)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_knn, average='macro')
roc_auc = roc_auc_score(y_test, y_pred_knn)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_knn)
classification = metrics.classification_report(y_test, y_pred_knn)
print()
print('============================== {} - AVALIAÇÃO - KNN ==============================')
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - KNN ==============================

PRECISãO:
 0.4913294797687861

RECALL:
 0.8673469387755102

F1:
 0.6273062730627306

ACURÁRIA BALANCEADA:
 0.9328996933607432

ACURÁCIA:
 0.9982268881008391

PRECISãO MACRO:
 0.7455502810983605

RECALL MACRO:
 0.9328996933607432

F1 MACRO:
 0.8132088015820018

ROC/AUC:
 0.9328996933607433

MATRIZ DE CONFUSÃO:
 [[56776    88]
 [   13    85]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.49      0.87      0.63        98

    accuracy                           1.00     56962
   macro avg       0.75      0.93      0.81     56962
weighted avg       1.00      1.00      1.00     56962




In [68]:
precision = precision_score(y_test, y_pred_rf)
recall = recall_score(y_test, y_pred_rf)
f1 = f1_score(y_test, y_pred_rf)
ballance_accuracy = balanced_accuracy_score(y_test, y_pred_rf)
accuracy = accuracy_score(y_test, y_pred_rf)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_rf, average='macro')
roc_auc = roc_auc_score(y_test, y_pred_rf)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_rf)
classification = metrics.classification_report(y_test, y_pred_rf)
print()
print('============================== {} - AVALIAÇÃO - RANDOM FOREST ==============================')
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - RANDOM FOREST ==============================

PRECISãO:
 0.3702127659574468

RECALL:
 0.8877551020408163

F1:
 0.5225225225225225

ACURÁRIA BALANCEADA:
 0.9425762004295246

ACURÁCIA:
 0.9972086654260736

PRECISãO MACRO:
 0.6850094273843856

RECALL MACRO:
 0.9425762004295246

F1 MACRO:
 0.760561381869408

ROC/AUC:
 0.9425762004295246

MATRIZ DE CONFUSÃO:
 [[56716   148]
 [   11    87]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.37      0.89      0.52        98

    accuracy                           1.00     56962
   macro avg       0.69      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962




In [69]:
precision = precision_score(y_test, y_pred_xgb)
recall = recall_score(y_test, y_pred_xgb)
f1 = f1_score(y_test, y_pred_xgb)
ballance_accuracy = balanced_accuracy_score(y_test, y_pred_xgb)
accuracy = accuracy_score(y_test, y_pred_xgb)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_xgb, average='macro')
roc_auc = roc_auc_score(y_test, y_pred_xgb)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_xgb)
classification = metrics.classification_report(y_test, y_pred_xgb)
print()
print('============================== {} - AVALIAÇÃO - XGB BOOST ==============================')
print()
print ("PRECISãO:" "\n", precision)
print()
print ("RECALL:" "\n", recall)
print()
print ("F1:" "\n", f1)
print()
print ("ACURÁRIA BALANCEADA:" "\n", ballance_accuracy)
print()
print ("ACURÁCIA:" "\n", accuracy)
print()
print ("PRECISãO MACRO:" "\n", precision_macro)
print()
print ("RECALL MACRO:" "\n", recall_macro)
print()
print ("F1 MACRO:" "\n", f1_macro)
print()
print ("ROC/AUC:" "\n", roc_auc)
print()
print("MATRIZ DE CONFUSÃO:" "\n", confusion_matrix)
print()
print("CLASSIFICAÇÃO:" "\n", classification) 
print()


============================== {} - AVALIAÇÃO - XGB BOOST ==============================

PRECISãO:
 0.09128205128205129

RECALL:
 0.9081632653061225

F1:
 0.1658900279589935

ACURÁRIA BALANCEADA:
 0.9462911149265558

ACURÁCIA:
 0.9842877707945648

PRECISãO MACRO:
 0.5455606498395003

RECALL MACRO:
 0.9462911149265558

F1 MACRO:
 0.5789796082675402

ROC/AUC:
 0.9462911149265558

MATRIZ DE CONFUSÃO:
 [[55978   886]
 [    9    89]]

CLASSIFICAÇÃO:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.09      0.91      0.17        98

    accuracy                           0.98     56962
   macro avg       0.55      0.95      0.58     56962
weighted avg       1.00      0.98      0.99     56962


